In [5]:
#!/usr/bin/env python
# coding: utf-8


import pandas as pd 
import numpy as np 
from mergeStock import merge, mergeYahoo
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def dataProcessing(stock,scaler_choice='MinMax', split=0.8, features='customized'):
    df = mergeYahoo(stock)
    # define input variable
    df['leading_sentiment'] = (df.pos_sum.abs() > df.neg_sum.abs())
    if features =='customized':
        X = ['popularity_daily_avg','popularity_daily_change', 
             'Open', 'Close','Volume', 'returnClosePrev1', 'returnClosePrev5', 'returnOpenPrev1', 'returnOpenPrev5',
             'vix_returnPrev1',
             'vader_mean', 'vader_std','comment_count', 'pos_mean', 'neg_mean','leading_sentiment']
    if features =='all':
        X = df.columns.tolist()
        X.remove('Y')
    # define target variable
    Y = ['Y']
    df[Y] = df[Y].astype(int)
    
    # train test split 
    train = df[:int(len(df)*split)]
    test = df[int(len(df)*split):]
    
    # MinMax Scaler
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    
    if scaler_choice == 'MinMax':
        scaler = MinMaxScaler()
        train[X] = scaler.fit_transform(train[X])
        test[X] = scaler.transform(test[X])
    if scaler_choice == 'Smooth':
        scaler = MinMaxScaler()
            # Train the Scaler with training data & smooth data
        smoothing_window_size = 90
        for di in range(0,len(train),smoothing_window_size):
            try:
                scaler.fit(train[X][di:di+smoothing_window_size])
                train.loc[train.index[di]:train.index[di+smoothing_window_size],X] = scaler.transform(train.loc[train.index[di]:train.index[di+smoothing_window_size],X])
            except:
                break
        # normalize the last bit
        print(di)
        scaler.fit(train[X][di:])
    #         train[X][di+smoothing_window_size:] = scaler.transform(train[X][di+smoothing_window_size:])
        train.loc[train.index[di]:,X] = scaler.transform(train.loc[train.index[di]:,X])
        test[X] = scaler.transform(test[X])
        print('finished!')
    if scaler_choice == 'Standard':
        scaler = StandardScaler()
        train[X] = scaler.fit_transform(train[X])
        test[X] = scaler.transform(test[X])
    return train[X], train[Y].values, test[X], test[Y].values

def getTop30():
    df = pd.read_excel('/Users/shelly/Google Drive/DS4A - Team 23/data/top_30_summary_final.xlsx')
    stock_list = df.ticker.tolist()
    return stock_list

def main(stock):
    train_X, train_Y, test_X, test_Y = dataProcessing(stock)
    model = MLPClassifier(activation='relu', hidden_layer_sizes =(32,), max_iter= 700, random_state= 42, 
                          solver='adam')
    model.fit(train_X, train_Y)
    y_true, y_pred = test_Y, model.predict(test_X)
    acc_s = accuracy_score(y_true, y_pred)
    print(stock,acc_s)
    return (stock, acc_s)

stock_list = getTop30()
accuracy_list = []
for stock in stock_list:
    accuracy_list.append(main(stock))

[*********************100%***********************]  1 of 1 completed


/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['close_T-1'] = price_df.Close.shift(periods =1)
/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['Y']= price_df.apply(lambda x: 1 if (x['Close'] - x['close_T-1']) > 0 else 0, axis=1).shift(periods =-1)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a 

AAL 0.5929203539823009
[*********************100%***********************]  1 of 1 completed


/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['close_T-1'] = price_df.Close.shift(periods =1)
/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['Y']= price_df.apply(lambda x: 1 if (x['Close'] - x['close_T-1']) > 0 else 0, axis=1).shift(periods =-1)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a 

AAPL 0.48672566371681414
[*********************100%***********************]  1 of 1 completed


...

TWTR 0.504424778761062
[*********************100%***********************]  1 of 1 completed


/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['close_T-1'] = price_df.Close.shift(periods =1)
/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['Y']= price_df.apply(lambda x: 1 if (x['Close'] - x['close_T-1']) > 0 else 0, axis=1).shift(periods =-1)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a 

UBER 0.4603174603174603
[*********************100%***********************]  1 of 1 completed


/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['close_T-1'] = price_df.Close.shift(periods =1)
/Users/shelly/Google Drive/DS4A - Team 23/mergeStock.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['Y']= price_df.apply(lambda x: 1 if (x['Close'] - x['close_T-1']) > 0 else 0, axis=1).shift(periods =-1)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a 

ZNGA 0.4336283185840708


In [6]:
accuracy_list

[('AAL', 0.5929203539823009),
 ('AAPL', 0.48672566371681414),
 ('ACB', 0.5108695652173914),
 ('AMD', 0.5132743362831859),
 ('AMZN', 0.4247787610619469),
 ('BABA', 0.5929203539823009),
 ('BAC', 0.5398230088495575),
 ('CGC', 0.5545454545454546),
 ('CRON', 0.5663716814159292),
 ('DAL', 0.5752212389380531),
 ('DIS', 0.504424778761062),
 ('F', 0.5221238938053098),
 ('FB', 0.40707964601769914),
 ('FIT', 0.5575221238938053),
 ('GE', 0.504424778761062),
 ('GPRO', 0.5221238938053098),
 ('HEXO', 0.5454545454545454),
 ('MSFT', 0.5309734513274337),
 ('NFLX', 0.45132743362831856),
 ('NIO', 0.43157894736842106),
 ('NKLA', 0.4),
 ('NVDA', 0.5398230088495575),
 ('PLUG', 0.4778761061946903),
 ('SBUX', 0.49557522123893805),
 ('SNAP', 0.49557522123893805),
 ('TOPS', 0.0),
 ('TSLA', 0.5752212389380531),
 ('TWTR', 0.504424778761062),
 ('UBER', 0.4603174603174603),
 ('ZNGA', 0.4336283185840708)]

In [7]:
accuracy_list.sort(key=lambda x: x[1], reverse = True)

In [8]:
accuracy_list

[('AAL', 0.5929203539823009),
 ('BABA', 0.5929203539823009),
 ('DAL', 0.5752212389380531),
 ('TSLA', 0.5752212389380531),
 ('CRON', 0.5663716814159292),
 ('FIT', 0.5575221238938053),
 ('CGC', 0.5545454545454546),
 ('HEXO', 0.5454545454545454),
 ('BAC', 0.5398230088495575),
 ('NVDA', 0.5398230088495575),
 ('MSFT', 0.5309734513274337),
 ('F', 0.5221238938053098),
 ('GPRO', 0.5221238938053098),
 ('AMD', 0.5132743362831859),
 ('ACB', 0.5108695652173914),
 ('DIS', 0.504424778761062),
 ('GE', 0.504424778761062),
 ('TWTR', 0.504424778761062),
 ('SBUX', 0.49557522123893805),
 ('SNAP', 0.49557522123893805),
 ('AAPL', 0.48672566371681414),
 ('PLUG', 0.4778761061946903),
 ('UBER', 0.4603174603174603),
 ('NFLX', 0.45132743362831856),
 ('ZNGA', 0.4336283185840708),
 ('NIO', 0.43157894736842106),
 ('AMZN', 0.4247787610619469),
 ('FB', 0.40707964601769914),
 ('NKLA', 0.4),
 ('TOPS', 0.0)]